<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span><ul class="toc-item"><li><span><a href="#Использование-техники-upsampling" data-toc-modified-id="Использование-техники-upsampling-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Использование техники upsampling</a></span></li><li><span><a href="#Использование-параметра-class_weight='balanced'" data-toc-modified-id="Использование-параметра-class_weight='balanced'-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Использование параметра class_weight='balanced'</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span><ul class="toc-item"><li><span><a href="#1.-Подготовка-данных" data-toc-modified-id="1.-Подготовка-данных-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span><b>1. Подготовка данных</b></a></span></li><li><span><a href="#2.-Исследование-задачи" data-toc-modified-id="2.-Исследование-задачи-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span><b>2. Исследование задачи</b></a></span></li><li><span><a href="#3.-Борьба-с-дисбалансом" data-toc-modified-id="3.-Борьба-с-дисбалансом-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span><b>3. Борьба с дисбалансом</b></a></span></li><li><span><a href="#4.-Тестирование-модели" data-toc-modified-id="4.-Тестирование-модели-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span><b>4. Тестирование модели</b></a></span></li></ul></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [33]:
# Импортирую нужные библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
!pip install imblearn
from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

from sklearn.metrics import (
    f1_score, 
    roc_auc_score, 
    confusion_matrix
)
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

In [2]:
# Смотрю данные
data = pd.read_csv('/datasets/Churn.csv')

data.info();

data.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


В данных отсутствуют пропуски, кроме Tenure, что позволяет сразу перейти к работе с ними. 

Exited - категориальный признак, поэтому нужно решить задачу классификации.

In [3]:
data = data.dropna()

In [4]:
# Убираю столбцы с номером строки, id клиента и фамилией, потому что они не пригодятся в обучении моделей

target = data['Exited']
features = data.drop(['RowNumber', 'CustomerId', 'Surname', 'Exited'], axis=1)

# Использую технику прямого кодирования (One-Hot Encoding). Чтобы не попасть в дамми-ловушку, применяю аргумент drop_first функции pd.get_dummies()..

features = pd.get_dummies(features, drop_first=True)

# Выделяю валидационную выборку

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

# Выделяю тестовую выборку из оставшихся тренировочных данных

features_train, features_test, target_train, target_test = train_test_split(
    features_train, target_train, test_size=0.25, random_state=12345)

In [5]:
# Стандартизирую признаки

scaler = StandardScaler()
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary']
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

In [6]:
print('Размер тренировочной выборки:',features_train.shape)
print()
print('Размер валидационной выборки:',features_valid.shape)
print()
print('Размер тестовой выборки:',features_test.shape)
print()
features.head(10)

Размер тренировочной выборки: (5454, 11)

Размер валидационной выборки: (1819, 11)

Размер тестовой выборки: (1818, 11)



,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,1,0
5,645,44,8.0,113755.78,2,1,0,149756.71,0,1,1
6,822,50,7.0,0.00,2,1,1,10062.80,0,0,1
7,376,29,4.0,115046.74,4,1,0,119346.88,1,0,0
8,501,44,4.0,142051.07,2,0,1,74940.50,0,0,1
9,684,27,2.0,134603.88,1,1,1,71725.73,0,0,1


In [7]:
data.groupby('Exited')['CreditScore'].count()

Exited
0    7237
1    1854
Name: CreditScore, dtype: int64

<b>Итоги подготовки данных</b>

В данных отсутствуют пропуски, кроме Tenure, что позволяет сразу перейти к работе с ними.

Exited - категориальный признак, поэтому нужно было решить задачу классификации.

Для корректного обучения модели стандартизировал следующие признаки:

   - Кредитный рейтинг('CreditScore')
   - Возраст ('Age')
   - Возраст ('Tenure')
   - Баланс на счёте ('Balance')
   - Предполагаемая зарплата ('EstimatedSalary')

Для корректного обучения модели следующие признаки не учитывались:

 - индекс строки в данных ('RowNumber')
 - уникальный идентификатор клиента ('CustomerId')
 - фамилия ('Surname')
 
Для лучшего обучения моделей была использована техника прямого кодирования (One-Hot Encoding).

Данные были разделены на выборки:
 - Тренировочную (60%)
 - Валидационную (20%)
 - Тестовую (20%)

В данных наблюдается значательный перевес в пользу отрицательного класса целевого признака. Положительного класса в четыре раза меньше, чем отрицательного.

 - Объём отрицательного класса: 7237
 - Объём положительного класса: 1854


## Исследование задачи

In [8]:
# Применяю модель "логистическая регрессия" и смотрю на её показатели без учёта дисбаланса

model_1 = LogisticRegression(random_state=12345, max_iter=1000, solver='lbfgs') 
model_1.fit(features_train, target_train)
predicted_valid_model_1 = model_1.predict(features_valid)

print('F1-мера модели LogisticRegression:', f1_score(target_valid, predicted_valid_model_1))

probabilities_valid = model_1.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc_model_1 = roc_auc_score(target_valid, probabilities_one_valid)

print()
print('Показатель AUC-ROC модели LogisticRegression на валидационной выборке:', auc_roc_model_1)

F1-мера модели LogisticRegression: 0.32653061224489793

Показатель AUC-ROC модели LogisticRegression на валидационной выборке: 0.7881824128586113


In [37]:
confusion_matrix(y_pred=predicted_valid_model_1, y_true=target_valid, normalize='true')

array([[0.72551724, 0.27448276],
       [0.2899729 , 0.7100271 ]])

In [9]:
# Применяю модель "дерево решений" и смотрю на её показатели без учёта дисбаланса

best_depth = 0
best_result = 0
for depth in range(1, 100):
    model_2 = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model_2.fit(features_train, target_train)
    predicted_valid_model_2 = model_2.predict(features_valid)
    result = f1_score(target_valid, predicted_valid_model_2)
    if result > best_result:
        best_result = result
        best_depth = depth

print('Лучшая F1-мера модели DecisionTreeClassifier:', best_result)
print('Оптимальная глубина дерева:', best_depth)

model_2 = DecisionTreeClassifier(random_state=12345, max_depth=best_depth) 
model_2.fit(features_train, target_train)

probabilities_valid = model_2.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc_model_2 = roc_auc_score(target_valid, probabilities_one_valid)

print()
print('Показатель AUC-ROC модели DecisionTreeClassifier на валидационной выборке:', auc_roc_model_2)

Лучшая F1-мера модели DecisionTreeClassifier: 0.5942684766214177
Оптимальная глубина дерева: 9

Показатель AUC-ROC модели DecisionTreeClassifier на валидационной выборке: 0.8037790860667227


In [36]:
model_2 = DecisionTreeClassifier(random_state=12345, max_depth=best_depth) 
model_2.fit(features_train, target_train)
predicted_valid_model_2 = model_2.predict(features_valid)

confusion_matrix(y_pred=predicted_valid_model_2, y_true=target_valid, normalize='true')

array([[0.93310345, 0.06689655],
       [0.46612466, 0.53387534]])

In [10]:
# Применяю модель "случайный лес" и смотрю на её показатели без учёта дисбаланса

best_depth = 0
best_est = 0
best_result = 0
for depth in range(1, 100, 5):
    for est in range(1, 100, 5):
        model_3_1 = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth ) 
        model_3_1.fit(features_train, target_train)
        predicted_valid_model_3_1 = model_3_1.predict(features_valid)
        result = f1_score(target_valid, predicted_valid_model_3_1)
        if result > best_result:
            best_result = result
            best_depth = depth
            best_est = est

print('Лучшая F1-мера модели RandomForestClassifier:', best_result)
print('Оптимальная глубина дерева:', best_depth)
print('Оптимальное количество деревьев:', best_est)

model_3_2 = RandomForestClassifier(random_state=12345, n_estimators=best_est, max_depth=best_depth ) 
model_3_2.fit(features_train, target_train)

probabilities_valid = model_3_2.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc_model_3_2 = roc_auc_score(target_valid, probabilities_one_valid)

print()
print('Показатель AUC-ROC модели RandomForestClassifier на валидационной выборке:', auc_roc_model_3_2)

Лучшая F1-мера модели RandomForestClassifier: 0.5621805792163544
Оптимальная глубина дерева: 26
Оптимальное количество деревьев: 71

Показатель AUC-ROC модели RandomForestClassifier на валидационной выборке: 0.8534875245304175


In [35]:
model_3_2 = RandomForestClassifier(random_state=12345, n_estimators=best_est, max_depth=best_depth ) 
model_3_2.fit(features_train, target_train)
predicted_valid_model_3_2 = model_3_2.predict(features_valid)

confusion_matrix(y_pred=predicted_valid_model_3_2, y_true=target_valid, normalize='true')

array([[0.97310345, 0.02689655],
       [0.59349593, 0.40650407]])

In [34]:
model_dummy = DummyClassifier(random_state=12345)
model_dummy.fit(features_train, target_train)

predicted_valid_model_dummy = model_dummy.predict(features_valid)

confusion_matrix(y_pred=predicted_valid_model_dummy, y_true=target_valid, normalize='true')

array([[1., 0.],
       [1., 0.]])

Результаты настройки моделей классификации и подбора оптимальных параметров <b>без учёта дисбаланса</b>:

<b>Показатели F1-меры без учёта дисбаланса:</b>
 
  - Моделью с самой высокой F1-мерой является дерево решений (DecisionTreeClassifier). Его F1-мера составляет <b>0.5942684766214177</b> при глубине дерева 9. 
 

  - На втором месте по значению F1-меры - случайный лес (RandomForestClassifier). Его F1-мера <b>0.5621805792163544</b>. При числе деревьев (n_estimators) 71 и глубине (max_depth) 26, эти парметры являются оптимальными.
 

  - На третьем месте логистическая регрессия (LogisticRegression) с F1-мерой <b>0.32653061224489793</b>.
  
<b>Показатели AUC-ROC без учёта дисбаланса:</b>

 - Самое высокое значение AUC-ROC на валидационной выборке у случайного леса (RandomForestClassifier) - <b>0.8534875245304175</b>
 
 
 - На втором месте по значению AUC-ROC - дерево решений (DecisionTreeClassifier). Его показатель AUC-ROC на валидационной выборке <b>0.8037790860667227</b> 
 
 
 - На третьем месте логистическая регрессия (LogisticRegression) с показатель AUC-ROC на валидационной выборке <b>0.7881824128586113</b>.
 
При проверке моделей на адекватность с помощбю «дамми-классифайера» виден перекос в пользу положительного класса. Ложно положительных ответов значительно больше, чем отрицательных, у случайного леса и дерева решений. Самой адекватной моделью оказалась логистическая регрессия.

## Борьба с дисбалансом

### Использование техники upsampling

Пробую использовать технику upsampling для преодоления дисбаланса.

In [11]:
# Создаю новую тренировочную выборку

oversample = SMOTE(random_state=12345)

features_upsampled, target_upsampled = oversample.fit_resample(features_train, target_train)

In [12]:
# Обучаю модель "логистическая регрессия" на новой тренировочной выборке и смотрю её показатели F1-меры и AUC-ROC

model_1 = LogisticRegression(random_state=12345, max_iter=1000, solver='lbfgs') 
model_1.fit(features_upsampled, target_upsampled)
predicted_valid_model_1 = model_1.predict(features_valid)

print('F1-мера модели LogisticRegression:', f1_score(target_valid, predicted_valid_model_1))

probabilities_valid = model_1.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc_model_1 = roc_auc_score(target_valid, probabilities_one_valid)

print()
print('Показатель AUC-ROC модели LogisticRegression на валидационной выборке:', auc_roc_model_1)

F1-мера модели LogisticRegression: 0.5070422535211268

Показатель AUC-ROC модели LogisticRegression на валидационной выборке: 0.7823829548640314


In [13]:
# Обучаю модель "дерево решений" на новой тренировочной выборке и смотрю её показатели F1-меры и AUC-ROC

best_depth = 0
best_result = 0
for depth in range(1, 100):
    model_2 = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model_2.fit(features_upsampled, target_upsampled)
    predicted_valid_model_2 = model_2.predict(features_valid)
    result = f1_score(target_valid, predicted_valid_model_2)
    if result > best_result:
        best_result = result
        best_depth = depth

print('Лучшая F1-мера модели DecisionTreeClassifier:', best_result)
print('Оптимальная глубина дерева:', best_depth)

model_2 = DecisionTreeClassifier(random_state=12345, max_depth=best_depth) 
model_2.fit(features_upsampled, target_upsampled)

probabilities_valid = model_2.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc_model_2 = roc_auc_score(target_valid, probabilities_one_valid)

print()
print('Показатель AUC-ROC модели DecisionTreeClassifier на валидационной выборке:', auc_roc_model_2)

Лучшая F1-мера модели DecisionTreeClassifier: 0.5445665445665445
Оптимальная глубина дерева: 8

Показатель AUC-ROC модели DecisionTreeClassifier на валидационной выборке: 0.8004158489860762


In [14]:
# Обучаю модель "случайный лес" на новой тренировочной выборке и смотрю её показатели F1-меры и AUC-ROC

best_depth = 0
best_est = 0
best_result = 0
for depth in range(1, 100, 5):
    for est in range(1, 100, 5):
        model_3_1 = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth ) 
        model_3_1.fit(features_upsampled, target_upsampled)
        predicted_valid_model_3_1 = model_3_1.predict(features_valid)
        result = f1_score(target_valid, predicted_valid_model_3_1)
        if result > best_result:
            best_result = result
            best_depth = depth
            best_est = est

print('Лучшая F1-мера модели RandomForestClassifier:', best_result)
print('Оптимальная глубина дерева:', best_depth)
print('Оптимальное количество деревьев:', best_est)

model_3_2 = RandomForestClassifier(random_state=12345, n_estimators=best_est, max_depth=best_depth ) 
model_3_2.fit(features_upsampled, target_upsampled)

probabilities_valid = model_3_2.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc_model_3_2 = roc_auc_score(target_valid, probabilities_one_valid)

print()
print('Показатель AUC-ROC модели RandomForestClassifier на валидационной выборке:', auc_roc_model_3_2)

Лучшая F1-мера модели RandomForestClassifier: 0.6277712952158694
Оптимальная глубина дерева: 6
Оптимальное количество деревьев: 61

Показатель AUC-ROC модели RandomForestClassifier на валидационной выборке: 0.8602392299785065


### Использование параметра class_weight='balanced'

Для того, чтобы сбалансировать классы, я добавил в параметры моделей class_weight='balanced'. Это помогло значительно повысить показатели  F1-меры. Этот метод оказался эффективнее техники upsampling.

In [15]:
# Применяю модель "логистическая регрессия" и смотрю на её показатели с учётом дисбаланса

model_1 = LogisticRegression(random_state=12345, max_iter=1000, solver='lbfgs', class_weight='balanced') 
model_1.fit(features_train, target_train)
predicted_valid_model_1 = model_1.predict(features_valid)

print('F1-мера модели LogisticRegression:', f1_score(target_valid, predicted_valid_model_1))

probabilities_valid = model_1.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc_model_1 = roc_auc_score(target_valid, probabilities_one_valid)

print()
print('Показатель AUC-ROC модели LogisticRegression на валидационной выборке:', auc_roc_model_1)

F1-мера модели LogisticRegression: 0.5092322643343051

Показатель AUC-ROC модели LogisticRegression на валидационной выборке: 0.7880796187272219


In [16]:
# Применяю модель "дерево решений" и смотрю на её показатели с учётом дисбаланса

best_depth = 0
best_result = 0
for depth in range(1, 100):
    model_2 = DecisionTreeClassifier(random_state=12345, class_weight='balanced', max_depth=depth) 
    model_2.fit(features_train, target_train)
    predicted_valid_model_2 = model_2.predict(features_valid)
    result = f1_score(target_valid, predicted_valid_model_2)
    if result > best_result:
        best_result = result
        best_depth = depth

print('Лучшая F1-мера модели DecisionTreeClassifier:', best_result)
print('Оптимальная глубина дерева:', best_depth)

model_2 = DecisionTreeClassifier(random_state=12345, class_weight='balanced', max_depth=6) 
model_2.fit(features_train, target_train)

probabilities_valid = model_2.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc_model_2 = roc_auc_score(target_valid, probabilities_one_valid)

print()
print('Показатель AUC-ROC модели DecisionTreeClassifier на валидационной выборке:', auc_roc_model_2)

Лучшая F1-мера модели DecisionTreeClassifier: 0.5741728922091781
Оптимальная глубина дерева: 6

Показатель AUC-ROC модели DecisionTreeClassifier на валидационной выборке: 0.8272096065788244


In [20]:
# Применяю модель "случайный лес" и смотрю на её показатели с учётом дисбаланса

best_depth = 0
best_est = 0
best_result = 0
for depth in range(1, 100, 2):
    for est in range(1, 100, 2):
        model_3_1 = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, class_weight='balanced') 
        model_3_1.fit(features_train, target_train)
        predicted_valid_model_3_1 = model_3_1.predict(features_valid)
        result = f1_score(target_valid, predicted_valid_model_3_1)
        if result > best_result:
            best_result = result
            best_depth = depth
            best_est = est

print('Лучшая F1-мера модели RandomForestClassifier:', best_result)
print('Оптимальная глубина дерева:', best_depth)
print('Оптимальное количество деревьев:', best_est)

model_3_2 = RandomForestClassifier(random_state=12345, n_estimators=best_est, max_depth=best_depth, class_weight='balanced') 
model_3_2.fit(features_train, target_train)

probabilities_valid = model_3_2.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc_model_3_2 = roc_auc_score(target_valid, probabilities_one_valid)

print()
print('Показатель AUC-ROC модели RandomForestClassifier на валидационной выборке:', auc_roc_model_3_2)

Лучшая F1-мера модели RandomForestClassifier: 0.619718309859155
Оптимальная глубина дерева: 9
Оптимальное количество деревьев: 31

Показатель AUC-ROC модели RandomForestClassifier на валидационной выборке: 0.8550359779459864


Результаты настройки моделей классификации и подбора оптимальных параметров <b>с учётом дисбаланса</b> с использованием параметра class_weight='balanced':

<b>Показатели F1-меры:</b>
 
  - Моделью с самой высокой F1-мерой является случайный лес (RandomForestClassifier). Его F1-мера составляет <b>0.619718309859155</b>. При числе деревьев (n_estimators) 31 и глубине (max_depth) 9, эти парметры являются оптимальными. Однако благодаря технике upsampling удалось добиться еще более высоких показателей F1-меры - <b>0.6277712952158694</b> (при количестве деревьев 61 и глубине деревьев 6)
 

  - На втором месте по значению F1-меры - дерево решений (DecisionTreeClassifier). Его F1-мера <b>0.5741728922091781</b> при глубине дерева 6.
 

  - На третьем месте логистическая регрессия (LogisticRegression) с F1-мерой <b>0.5092322643343051</b>.
  
<b>Показатели AUC-ROC:</b>

 - Самое высокое значение AUC-ROC на валидационной выборке у случайного леса (RandomForestClassifier) - <b>0.8550359779459864</b>
 
 
 - На втором месте по значению AUC-ROC - дерево решений (DecisionTreeClassifier). Его показатель AUC-ROC на валидационной выборке <b>0.8272096065788244</b> 
 
 
 - На третьем месте логистическая регрессия (LogisticRegression) с показатель AUC-ROC на валидационной выборке <b>0.7880796187272219</b>.
 
С коррекцией моделей с учётом дисбаланса у случайного леса (RandomForestClassifier) значительно повысились показатели F1-меры и AUC-ROC. По показателям F1-меры случайный лес, таким образом, оказался лучшей моделью. При этом на тестовой выборке более высокие показатели F1-меры были достигнуты благодаря использованию параметра <b>class_weight='balanced'</b>, а не техники <b>upsampling</b>.

Показатель F1-меры с учётом дисбаланса у логистической регрессии (LogisticRegression) увеличился больше всего, при этом показатель AUC-ROC практически не изменился.

Показатель F1-меры с учётом дисбаланса у дерева решений (DecisionTreeClassifier) наоборот уменьшился, но при этом увеличилось значение AUC-ROC.

Использование параметра <b>class_weight='balanced'</b> для моделей дерева решений (DecisionTreeClassifier) и логистической регрессии (LogisticRegression) даёт более эффективный результат для увеличения метрики F1-меры, техника upsampling в данном случае менее эффективна.

Техника <b>upsampling</b> оказалась более эффективна, чем использование параметра <b>class_weight='balanced'</b>, для модели случайного леса (RandomForestClassifier) на валидационной выборке, но не на тестовой.

## Тестирование модели

In [26]:
model_3_2 = RandomForestClassifier(random_state=12345, n_estimators=31, class_weight='balanced', max_depth=9 ) 
model_3_2.fit(features_train, target_train)
predicted_test_model_3_2 = model_3_2.predict(features_test)
result = f1_score(target_test, predicted_test_model_3_2)
print('F1-мера модели RandomForestClassifier на тестовой выборке:', result)

probabilities_test = model_3_2.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc_model_3_2 = roc_auc_score(target_test, probabilities_one_test)

print('Показатель AUC-ROC модели RandomForestClassifier на тестовой выборке:', auc_roc_model_3_2)

F1-мера модели RandomForestClassifier на тестовой выборке: 0.5981554677206852
Показатель AUC-ROC модели RandomForestClassifier на тестовой выборке: 0.8496740881241627


Результаты проверки лучшей модели, случайного леса (RandomForestClassifier), на тестовой выборке:

 - <b>F1-мера</b> модели RandomForestClassifier на тестовой выборке: <b>0.5981554677206852</b>
 
 
 - Показатель <b>AUC-ROC</b> модели RandomForestClassifier на тестовой выборке: <b>0.8496740881241627</b>
    

## Вывод

В работе с данными и обучении модели были сделаны следующие шаги:

    1. Подготовка данных
    2. Исследование задачи
    3. Борьба с дисбалансом
    4. Тестирование модели

### <b>1. Подготовка данных</b>

В данных отсутствуют пропуски, кроме Tenure, что позволяет сразу перейти к работе с ними.

Exited - категориальный признак, поэтому нужно было решить задачу классификации.

Для корректного обучения модели стандартизировал следующие признаки:

   - Кредитный рейтинг('CreditScore')
   - Возраст ('Age')
   - Возраст ('Tenure')
   - Баланс на счёте ('Balance')
   - Предполагаемая зарплата ('EstimatedSalary')

Для корректного обучения модели следующие признаки не учитывались:

 - индекс строки в данных ('RowNumber')
 - уникальный идентификатор клиента ('CustomerId')
 - фамилия ('Surname')
 
Для лучшего обучения моделей была использована техника прямого кодирования (One-Hot Encoding).

Данные были разделены на выборки:
 - Тренировочную (60%)
 - Валидационную (20%)
 - Тестовую (20%)

В данных наблюдается значательный перевес в пользу отрицательного класса целевого признака. Положительного класса в четыре раза меньше, чем отрицательного.

 - Объём отрицательного класса: 7237
 - Объём положительного класса: 1854

### <b>2. Исследование задачи</b>

Результаты настройки моделей классификации и подбора оптимальных параметров <b>без учёта дисбаланса</b>:

<b>Показатели F1-меры без учёта дисбаланса:</b>
 
  - Моделью с самой высокой F1-мерой является дерево решений (DecisionTreeClassifier). Его F1-мера составляет <b>0.5942684766214177</b> при глубине дерева 9. 
 

  - На втором месте по значению F1-меры - случайный лес (RandomForestClassifier). Его F1-мера <b>0.5621805792163544</b>. При числе деревьев (n_estimators) 71 и глубине (max_depth) 26, эти парметры являются оптимальными.
 

  - На третьем месте логистическая регрессия (LogisticRegression) с F1-мерой <b>0.32653061224489793</b>.
  
<b>Показатели AUC-ROC без учёта дисбаланса:</b>

 - Самое высокое значение AUC-ROC на валидационной выборке у случайного леса (RandomForestClassifier) - <b>0.8534875245304175</b>
 
 
 - На втором месте по значению AUC-ROC - дерево решений (DecisionTreeClassifier). Его показатель AUC-ROC на валидационной выборке <b>0.8037790860667227</b> 
 
 
 - На третьем месте логистическая регрессия (LogisticRegression) с показатель AUC-ROC на валидационной выборке <b>0.7881824128586113</b>.
 
Результаты настройки моделей классификации и подбора оптимальных параметров <b>без учёта дисбаланса</b>:

<b>Показатели F1-меры без учёта дисбаланса:</b>
 
  - Моделью с самой высокой F1-мерой является дерево решений (DecisionTreeClassifier). Его F1-мера составляет <b>0.5942684766214177</b> при глубине дерева 9. 
 

  - На втором месте по значению F1-меры - случайный лес (RandomForestClassifier). Его F1-мера <b>0.5621805792163544</b>. При числе деревьев (n_estimators) 71 и глубине (max_depth) 26, эти парметры являются оптимальными.
 

  - На третьем месте логистическая регрессия (LogisticRegression) с F1-мерой <b>0.32653061224489793</b>.
  
<b>Показатели AUC-ROC без учёта дисбаланса:</b>

 - Самое высокое значение AUC-ROC на валидационной выборке у случайного леса (RandomForestClassifier) - <b>0.8534875245304175</b>
 
 
 - На втором месте по значению AUC-ROC - дерево решений (DecisionTreeClassifier). Его показатель AUC-ROC на валидационной выборке <b>0.8037790860667227</b> 
 
 
 - На третьем месте логистическая регрессия (LogisticRegression) с показатель AUC-ROC на валидационной выборке <b>0.7881824128586113</b>.
 
При проверке моделей на адекватность с помощбю «дамми-классифайера» виден перекос в пользу положительного класса. Ложно положительных ответов значительно больше, чем отрицательных, у случайного леса и дерева решений. Самой адекватной моделью оказалась логистическая регрессия.
 
### <b>3. Борьба с дисбалансом</b>

Результаты настройки моделей классификации и подбора оптимальных параметров <b>с учётом дисбаланса</b> с использованием параметра class_weight='balanced':

<b>Показатели F1-меры:</b>
 
  - Моделью с самой высокой F1-мерой является случайный лес (RandomForestClassifier). Его F1-мера составляет <b>0.619718309859155</b>. При числе деревьев (n_estimators) 31 и глубине (max_depth) 9, эти парметры являются оптимальными. Однако благодаря технике upsampling удалось добиться еще более высоких показателей F1-меры - <b>0.6277712952158694</b> (при количестве деревьев 61 и глубине деревьев 6)
 

  - На втором месте по значению F1-меры - дерево решений (DecisionTreeClassifier). Его F1-мера <b>0.5741728922091781</b> при глубине дерева 6.
 

  - На третьем месте логистическая регрессия (LogisticRegression) с F1-мерой <b>0.5092322643343051</b>.
  
<b>Показатели AUC-ROC:</b>

 - Самое высокое значение AUC-ROC на валидационной выборке у случайного леса (RandomForestClassifier) - <b>0.8550359779459864</b>
 
 
 - На втором месте по значению AUC-ROC - дерево решений (DecisionTreeClassifier). Его показатель AUC-ROC на валидационной выборке <b>0.8272096065788244</b> 
 
 
 - На третьем месте логистическая регрессия (LogisticRegression) с показатель AUC-ROC на валидационной выборке <b>0.7880796187272219</b>.
 
С коррекцией моделей с учётом дисбаланса у случайного леса (RandomForestClassifier) значительно повысились показатели F1-меры и AUC-ROC. По показателям F1-меры случайный лес, таким образом, оказался лучшей моделью. При этом на тестовой выборке более высокие показатели F1-меры были достигнуты благодаря использованию параметра <b>class_weight='balanced'</b>, а не техники <b>upsampling</b>.

Показатель F1-меры с учётом дисбаланса у логистической регрессии (LogisticRegression) увеличился больше всего, при этом показатель AUC-ROC практически не изменился.

Показатель F1-меры с учётом дисбаланса у дерева решений (DecisionTreeClassifier) наоборот уменьшился, но при этом увеличилось значение AUC-ROC.

Использование параметра <b>class_weight='balanced'</b> для моделей дерева решений (DecisionTreeClassifier) и логистической регрессии (LogisticRegression) даёт более эффективный результат для увеличения метрики F1-меры, техника upsampling в данном случае менее эффективна.

Техника <b>upsampling</b> оказалась более эффективна, чем использование параметра <b>class_weight='balanced'</b>, для модели случайного леса (RandomForestClassifier) на валидационной выборке, но не на тестовой.

### <b>4. Тестирование модели</b>

Результаты проверки лучшей модели, случайного леса (RandomForestClassifier), на тестовой выборке:

 - <b>F1-мера</b> модели RandomForestClassifier на тестовой выборке: <b>0.5981554677206852</b>
 
 
 - Показатель <b>AUC-ROC</b> модели RandomForestClassifier на тестовой выборке: <b>0.8496740881241627</b>